# Cifar-100 test for image classification

In [1]:
import sys
sys.path.insert(0, '../..')

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Resizing, Lambda, Flatten, Dense
import pandas as pd
import numpy as np
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

/Users/sasha/jax-metal/lib/python3.8/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Download DenseNet121 with Imagenet weights (1000 classes)

In [2]:
dnet = tf.keras.applications.densenet.DenseNet121(include_top=True,
                                                  weights='imagenet',
                                                  input_tensor=None,
                                                  input_shape=None,
                                                  pooling=None,
                                                  classes=1000,
                                                  classifier_activation='softmax'
                                                  )

In [3]:
dnet.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, 112, 112, 64)         9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1/conv[0][0]']

Make all layers untrainable except for the very last convolutional layer

In [4]:
for layer in dnet.layers:
    layer.trainable = False
dnet.layers[-6].trainable  = True

Cifar-100 testing

In [5]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

In [6]:
y_train_cat = to_categorical(y_train, 1000)
y_test_cat = to_categorical(y_test, 1000)

Lambda layer for preprocessing

In [7]:
def preprocess(x):
    x = tf.image.resize(x,size=(224,224),method='bicubic')
    x = tf.keras.applications.densenet.preprocess_input(x)
    return x

Modify the model

In [8]:
input_shape = (32,32,3)

In [9]:
input_layer = Input(shape=input_shape)
prep = Lambda(preprocess)(input_layer)
out = dnet(prep)
dnet_mod = Model(inputs=input_layer, outputs=out)

In [10]:
dnet_mod.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_categorical_accuracy')])

Try to fit it on Cifar-100 data and then evaluate (there is no hope this is gonna work ...)

In [11]:
dnet_mod.fit(X_train, y_train_cat)

1563/1563 [==============================] - 4034s 3s/step - loss: 9.6588 - top_1_categorical_accuracy: 0.0107


In [12]:
dnet_mod.evaluate(X_test, y_test_cat)

313/313 [==============================] - 672s 2s/step - loss: 9.2415 - top_1_categorical_accuracy: 0.0158


[9.241533279418945, 0.015799999237060547]

Try the same with adding a Cerebros "add-on" network

In [13]:
INPUT_SHAPES  = [input_shape]
OUTPUT_SHAPES = [100]

Use 10k-15k random samples from Cifar-100 to speed up the process

In [14]:
num_samples = 15_000
rng = np.random.default_rng()
ind = rng.permutation(X_train.shape[0])[:num_samples]

In [15]:
training_x   = [tf.constant(X_train[ind,:,:,:])]
y_train_cat  = to_categorical(y_train[ind], 100)
train_labels = [tf.constant(y_train_cat)]

In [16]:
dnet = tf.keras.applications.densenet.DenseNet121(include_top=True,
                                                  weights='imagenet',
                                                  input_tensor=None,
                                                  input_shape=None,
                                                  pooling=None,
                                                  classes=1000,
                                                  classifier_activation='softmax'
                                                  )

In [17]:
for layer in dnet.layers:
    layer.trainable = False
dnet.layers[-6].trainable  = True

In [18]:
dnet_io = Model(inputs=dnet.layers[0].input,
                outputs=dnet.layers[-2].output)

In [19]:
input_layer = Input(shape=input_shape)
prep = Lambda(preprocess)(input_layer)
out = Flatten()(dnet_io(prep))
base_mod = Model(inputs=input_layer, outputs=out)

In [20]:
activation = 'relu'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 10  # [1, 100]
batch_size = 20
maximum_levels = 4  # [3,7]
maximum_units_per_level = 7  # [2,10]
maximum_neurons_per_unit = 4  # [2,20]

In [21]:
# Final training task
TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
#
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test_cifar100_densenet'
#
meta_trial_number = 42
#
cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.2,
    direction='maximize',
    metric_to_rank_by="val_top_1_categorical_accuracy",
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation='softmax',
    number_of_architecture_moities_to_try=3,
    number_of_tries_per_architecture_moity=2,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.TopKCategoricalAccuracy(
                k=1, name='top_1_categorical_accuracy')
             ],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number,
    base_models=[base_mod])

In [22]:
%%time
result = cerebros_automl.run_random_search()

SimpleCerebrosRandomSearch.input_shapes: [(32, 32, 3)]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'1': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'4': <class 'cerebros.units.units.DenseUnit'>}, {'1': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}]
3
Trying to create level 3
We think level 3's predecessors are: [0, 1, 2]
k is: 4 value is: [{'100': <class 'cerebros.units.units.FinalDenseUnit'>}]
4
Trying to create Final l

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_1_dns_')>, <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_2_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_0_dns_')>, <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_2_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'Ne

INFO:tensorflow:Assets written to: 2023_10_22_14_00_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  1.724567                    0.544583  1.239056   
1  0.876107                    0.744833  1.213375   
2  0.635165                    0.806833  1.288063   
3  0.485002                    0.846250  1.338048   
4  0.379049                    0.879417  1.389307   
5  0.318090                    0.901000  1.465986   
6  0.268358                    0.910417  1.520408   
7  0.218966                    0.930250  1.639519   
8  0.208700                    0.931333  1.631307   
9  0.173237                    0.943417  1.731060   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.648667             0                0   
1                        0.667333             0                0   
2                        0.667333             0                0   
3                        0.654667             0                0   
4                        0.668333             0   

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000004_tr_0_FinalDenseUnit_0000000000000004_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_2_dns_')>, <KerasTensor: shape=(None, 1) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_1_dns_')>, <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_2_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000003_tr_0_DenseUnit_0000000000000003_tr_0_0_dns_')>, <KerasTensor: shape=(None, 4) dtype=float32 (created by layer 'Ne

...

INFO:tensorflow:Assets written to: 2023_10_22_14_00_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000000/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  1.459496                    0.612133  1.263111   
1  0.355672                    0.907083  1.441401   
2  0.148412                    0.965500  1.555182   
3  0.085637                    0.982583  1.641894   
4  0.072836                    0.983333  1.812616   
5  0.068504                    0.982417  1.999138   
6  0.079626                    0.976167  2.094380   
7  0.082244                    0.973083  2.258822   
8  0.078042                    0.974000  2.406696   
9  0.071021                    0.977750  2.491801   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.648000             0                0   
1                        0.645333             0                0   
2                        0.634333             0                0   
3                        0.645000             0                0   
4                        0.627333             0   

Debug: successor_connectivity_errors_2d []
materialize:_NeuralNetworkFuture_0000000000000nan_tr_0_FinalDenseLevel_0000000000000003_tr_0_FinalDenseUnit_0000000000000003_tr_0_0 called
materialized network layers
[<KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_0_dns_')>, <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_0_dns_')>, <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_0_dns_')>, <KerasTensor: shape=(None, 3) dtype=float32 (created by layer 'NeuralNetworkFuture_0000000000000nan_tr_0_DenseLevel_0000000000000002_tr_0_DenseUnit_0000000000000002_tr_0_0_dns_')>, <KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 

INFO:tensorflow:Assets written to: 2023_10_22_14_00_cerebros_auto_ml_test_cifar10_meta_42/models/tr_0000000000000000_subtrial_0000000000000001/assets


returning trial 0 oracles
       loss  top_1_categorical_accuracy  val_loss  \
0  1.404509                    0.622400  1.293745   
1  0.297108                    0.927500  1.429827   
2  0.109366                    0.981667  1.548144   
3  0.063623                    0.990583  1.698006   
4  0.050008                    0.990333  1.833229   
5  0.057087                    0.985083  2.073851   
6  0.072773                    0.979083  2.268977   
7  0.069746                    0.978500  2.341281   
8  0.067282                    0.977750  2.534348   
9  0.062089                    0.978833  2.710763   

   val_top_1_categorical_accuracy  trial_number  subtrial_number  \
0                        0.637000             0                1   
1                        0.643667             0                1   
2                        0.640667             0                1   
3                        0.636667             0                1   
4                        0.632333             0   

In [23]:
print(f'Best accuracy achieved is {result}')
print(f'top-1 categorical accuracy')

Best accuracy achieved is 0.67166668176651
top-1 categorical accuracy


Evaluating the best model found

In [24]:
best_model_found = cerebros_automl.get_best_model()

In [25]:
#
eval_loss = tf.keras.losses.CategoricalCrossentropy()
#
eval_metrics =\
[tf.keras.metrics.TopKCategoricalAccuracy(k=1,\
            name='eval_top_1_categorical_accuracy'),
 tf.keras.metrics.TopKCategoricalAccuracy(k=5,\
            name='eval_top_5_categorical_accuracy')
]

In [26]:
best_model_found.compile(loss=eval_loss, metrics=eval_metrics)
best_model_found.summary()

Model: "NeuralNetworkFuture_0000000000000nan_tr_0_nn_materialized"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 NeuralNetworkFuture_000000  [(None, 32, 32, 3)]          0         []                            
 0000000nan_tr_0_InputLevel                                                                       
 _0000000000000000_tr_0_Inp                                                                       
 utUnit_0000000000000000_tr                                                                       
 _0_0_inp (InputLayer)                                                                            
                                                                                                  
 model_2 (Functional)        (None, 1024)                 7037504   ['NeuralNetworkFuture_00000000
                                          

In [28]:
print("Evaluating best model found ...")
print("Loss | Top-1 accuracy | Top-5 accuracy")
y_test_cat = to_categorical(y_test, 100)
best_model_found.evaluate(X_test, y_test_cat)

Evaluating best model found ...
Loss | Top-1 accuracy | Top-5 accuracy
313/313 [==============================] - 354s 1s/step - loss: 2.6881 - eval_top_1_categorical_accuracy: 0.6071 - eval_top_5_categorical_accuracy: 0.8615


[2.688145875930786, 0.6071000099182129, 0.8615000247955322]